# Create High-k Metal Gate Stack Tutorial

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate).


## 1. Configuration Parameters


In [6]:
# Global parameters
MAX_AREA = [200,200,260]  # Maximum area for strain matching
MAX_AREA_RATIO_TOL = 0.25  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.15  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.15  # Maximum length tolerance for strain matching
VACUUM = 10.0  # Final vacuum spacing in Angstroms

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    {
        # Silicon substrate
        "slab_params": {
            "miller_indices": (1, 0, 0),
            "thickness": 4,  # atomic layers
            "vacuum": VACUUM,  # Angstroms
            "use_orthogonal_z": True
        },
        "interface_distance": None  # No interface for substrate
    },
    {
        # SiO2 layer
        "slab_params": {
            "miller_indices": (1,0,0),
            "thickness": 3,
            "vacuum": VACUUM,
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to Si substrate
    },
    {
        # HfO2 layer
        "slab_params": {
            "miller_indices": (0,0,1),
            "thickness": 4,
            "vacuum": 0.5,
            "xy_supercell_matrix": [[1, 0], [0, 2]],
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to SiO2
    },
    {
        # TiN layer
        "slab_params": {
            "miller_indices": (0,0, 1),
            "thickness": 3,
            "vacuum": VACUUM,  # Add vacuum to final layer
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to HfO2
    }
]

INTERFACE_1_INDEX = 11


## 2. Environment Setup and Material Loading


In [7]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from mat3ra.made.tools.modify import translate_to_z_level
from utils.visualize import visualize_materials as visualize

# Load materials from Standata
materials_1 = [
    Material(Materials.get_by_name_first_match("SiO2")),  # SiO2
    Material(Materials.get_by_name_first_match("Silicon")),  # Si substrate
    ]
materials_2 = [
    Material(Materials.get_by_name_first_match("HfO2")),  # HfO2
    Material(Materials.get_by_name_first_match("TiN")),  # TiN
]


## 3. Create Si/SiO2 Interface

In [ ]:
from mat3ra.made.tools.build.slab import PymatgenSlabGeneratorParameters


film_params = STRUCTURE_PARAMS[1]
substrate_params = STRUCTURE_PARAMS[0]

film_config = SlabConfiguration(
    bulk=materials_1[1],
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=materials_1[0],
    **substrate_params["slab_params"]
)
params = PymatgenSlabGeneratorParameters(symmetrize = False)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

film_termination = film_terminations[0]
substrate_termination = substrate_terminations[0]


interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)



builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=ZSLStrainMatchingParameters(max_area=MAX_AREA[0], max_area_ratio_tol=MAX_AREA_RATIO_TOL, max_angle_tol=MAX_ANGLE_TOLERANCE, max_length_tol=MAX_LENGTH_TOLERANCE)
    )
)

interfaces = builder.get_materials(configuration=interface_config)

visualize(
    interfaces[11],
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface"
)

visualize(
    interfaces[11],
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface",
    rotation='-90x'
)

interface_1 = interfaces[INTERFACE_1_INDEX]





## 2. Add HfO2 Layer

In [ ]:
from mat3ra.made.tools.build.interface import SimpleInterfaceBuilder, SimpleInterfaceBuilderParameters

film_params = STRUCTURE_PARAMS[2]

film_config = SlabConfiguration(
    bulk=materials_2[0],
    **film_params["slab_params"]
)

hfo2_slab= create_slab(film_config, build_parameters=params)

film_config = SlabConfiguration(
    bulk=hfo2_slab,
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=interface_1,
    miller_indices=(0,0,1)
)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

print("Film terminations:", film_terminations)
print("Substrate terminations:", substrate_terminations)


interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_terminations[0],
    substrate_termination=substrate_terminations[0],
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))
interface = builder.get_material(configuration=interface_config)

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface"
)

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface",
    rotation='-90x'
)


## 3. Add TiN Layer

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell

film_params = STRUCTURE_PARAMS[3]

interface = create_supercell(interface, scaling_factor=[1,1,-1])

film_config = SlabConfiguration(
    bulk=materials_2[1],
    **film_params["slab_params"]
)

tin_slab = create_slab(film_config, build_parameters=params)

film_config = SlabConfiguration(
    bulk=tin_slab,
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=interface,
    miller_indices=(0,0,1)
)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

print("Film terminations:", film_terminations)
print("Substrate terminations:", substrate_terminations)

interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_terminations[0],
    substrate_termination=substrate_terminations[0],
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))

interface = builder.get_material(configuration=interface_config)

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface"
)

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface",
    rotation='-90x'
)

# invert the interface along the z-axis to match example
interface = create_supercell(interface, scaling_factor=[1,1,-1])

## 4. Final Structure Visualization and Analysis


In [ ]:
from utils.jupyterlite import set_materials

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface",
)

visualize(
    interface,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface",
    rotation='-90x'
)

set_materials(interface)